In [ ]:
import numpy as np
import pandas as pd
import h5py
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import os
from peakfinder import detect_peaks
from sklearn.cluster import DBSCAN
from tqdm import tqdm_notebook as tqdm
import similaritymeasures
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import math
from multiprocessing import Pool
plt.style.use('ggplot')

In [ ]:
df_labels = pd.read_csv(r'C:\Users\kj4755\OneDrive - The Open University\SPIN\Transmission\Scripts\new labels1.txt',delimiter = '\s+',header = None,index_col = None)
df_labels.columns = ['File','Label']


In [ ]:
z = []
y = []

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

f_wav = 250

rootDir = r'C:\Users\kj4755\OneDrive - The Open University\SPIN\data\level_1p0_data\New occultations'
#rootDir = r'/Users/kunal/OneDrive - The Open University/SPIN/data/level_1p0_data/New occultations'
os.chdir(rootDir)
list_of_files = np.array(os.listdir(os.getcwd()))


for each_file in tqdm(list_of_files):

    file = h5py.File(r'C:\Users\kj4755\OneDrive - The Open University\SPIN\data\level_1p0_data\New occultations\%s' %each_file,'r')
    #file = h5py.File(r'/Users/kunal/OneDrive - The Open University/SPIN/data/level_1p0_data/New occultations/%s' %each_file,'r')


    T = np.array(file['Science/Transmission'])
    TangAlt = np.array(file['Geometry/Point0/TangentAltSurface'])
    wav = np.array(file['Science/Wavelength'])

    avg_TangAlt = []

    for j in range(TangAlt.shape[0]):
        avg_TangAlt.append(np.mean(TangAlt[j,:]))

    T_wav = T[:,np.array(np.where(wav == find_nearest(wav,f_wav))).flatten()].reshape(-1,)

    if T_wav[0] > 0.5:
        T_wav = T_wav[::-1]

    z.append(T_wav)

    if avg_TangAlt[0] > 100:
        avg_TangAlt = avg_TangAlt[::-1]

    y.append(avg_TangAlt)
            
z = np.array(z)
y = np.array(y)

bad_files_idx = []
for i,_ in enumerate(z):
    if _[0] > 0.8:
        bad_files_idx.append(i)
z = np.delete(z,bad_files_idx,0)
y = np.delete(y,bad_files_idx,0)

In [ ]:
alt_interp = np.arange(0,max(np.array([len(_) for _ in z])),1)
t_interp = []
for i in range(len(z)):
    z_interp = np.interp(alt_interp,y[i],z[i])
    t_interp.append(z_interp)
t_interp = np.array(t_interp)

In [ ]:
t_clipped = []
for j,t in enumerate(t_interp):
    clip = np.array([])
    peak_idx = detect_peaks(t,mph=0.05)
    for _ in t[peak_idx]:
        clip = np.append(clip,_)
        if _ > 0.963:
            break
    t_clipped.append(t[:np.where(t == clip[-1])[0][0]])
t_clipped = np.array(t_clipped)           

In [ ]:
clipped_len = []
for t in t_clipped: 
    clipped_len.append(len(t))

t_clipped_max = t_interp[:,:max(clipped_len)]

In [ ]:
max(np.array([len(_) for _ in z]))

In [ ]:
for i,t in enumerate(t_clipped_max):
    
    peak_idx = detect_peaks(t,mph=0.05)
    valley_idx = detect_peaks(t,mph = 0.963, valley = True)

    p_structure_low = []
    p_structure_high = []
    v_structure_low = []
    v_structure_high = []
    
    for peak in peak_idx:
        if 0.05 < t[peak] <= 0.2:
            p_structure_low.append(peak)
        if 0.2 < t[peak] < 0.963:
            p_structure_high.append(peak)
            
    for valley in valley_idx:
        if 0.05 < t[valley] < 0.2:
            v_structure_low.append(valley)
        if t[valley] > 0.2:
            v_structure_high.append(valley)
        
            
    p_structure_high = np.array(p_structure_high)
    p_structure_low = np.array(p_structure_low)
    v_structure_low = np.array(v_structure_low)
    v_structure_high = np.array(v_structure_high)
    
    if len(p_structure_low) != 0:
        t_clipped_max[i,p_structure_low[0]] = 5
    
    if len(p_structure_high) != 0:
        t_clipped_max[i,p_structure_high[0]] = 10
    
    if len(v_structure_high) != 0:
        t_clipped_max[i,v_structure_high[0]] = 15
    

In [ ]:
alt_interp = np.arange(0,274,1)
t_interp = []
for i in range(len(z)):
    z_interp = np.interp(alt_interp,y[i],z[i])
    t_interp.append(z_interp)
t_interp = np.array(t_interp)

t_clipped = []
for j,t in enumerate(t_interp):
    clip = np.array([])
    peak_idx = detect_peaks(t)
    for _ in t[peak_idx]:
        clip = np.append(clip,_)
        if _ > 0.98:
            break
    t_clipped.append(t[:np.where(t == clip[-1])[0][0]])
t_clipped = np.array(t_clipped) 

In [ ]:
i = 380

plt.plot(t_clipped_max[i],'b')


peak_idx = detect_peaks(t_interp[i],mph=0.05)
valley_idx = detect_peaks(t_interp[i],mph = 0.963, valley = True)

p_structure_low = []
p_structure_high = []
v_structure_low = []
v_structure_high = []

for peak in peak_idx:
    if 0.05 < t_interp[i,peak] <= 0.2:
        p_structure_low.append(peak)
    if 0.2 < t_interp[i,peak] < 0.963:
        p_structure_high.append(peak)

for valley in valley_idx:
    if 0.05 < t_interp[i,valley] < 0.2:
        v_structure_low.append(valley)
    if t_interp[i,valley] > 0.2:
        v_structure_high.append(valley)


p_structure_high = np.array(p_structure_high)
p_structure_low = np.array(p_structure_low)
v_structure_low = np.array(v_structure_low)
v_structure_high = np.array(v_structure_high)
    
print(v_structure_high,v_structure_low)
print(p_structure_high,p_structure_low)


plt.plot(t_interp[i],'b--')       
        
if len(p_structure_low) != 0:
    plt.plot(p_structure_low,t_interp[i,p_structure_low],'rx')
    
if len(p_structure_high) != 0:
    plt.plot(p_structure_high,t_interp[i,p_structure_high],'kx')
    
if len(v_structure_high) != 0:
    plt.plot(v_structure_high,t_interp[i,v_structure_high],'bx')

In [ ]:
'''dm = np.zeros((len(t_clipped_max),len(t_clipped_max)))

for i in tqdm(range(len(t_clipped_max))):
    
    t0 = np.zeros((len(t_clipped_max[0]),2))
    t0[:,0] = np.arange(len(t_clipped_max[0]))
    t0[:,1] = t_clipped_max[i]
    
    for j in range(len(t_clipped_max)):
        t1 = np.zeros((len(t_clipped_max[0]),2))
        t1[:,0] = np.arange(len(t_clipped_max[0]))
        t1[:,1] = t_clipped_max[j]
        
        dm[i,j] =similaritymeasures.curve_length_measure(t0,t1)
        #dm[i,j] =similaritymeasures.dtw(t0,t1)[0]
        #dm[i,j] =fastdtw(t0,t1,dist = euclidean)[0]'''

'''a = t_clipped_max[:len(t_clipped_max)//3]
dm = np.zeros((len(a),len(a)))

for i in tqdm(range(len(a))):
    
    t0 = np.zeros((len(a[0]),2))
    t0[:,0] = np.arange(len(a[0]))
    t0[:,1] = a[i]
    
    for j in range(len(a)):
        t1 = np.zeros((len(a[0]),2))
        t1[:,0] = np.arange(len(a[0]))
        t1[:,1] = a[j]
        
        dm[i,j] =similaritymeasures.curve_length_measure(t0,t1)
        #dm[i,j] =similaritymeasures.dtw(t0,t1)[0]
        #dm[i,j] =fastdtw(t0,t1,dist = euclidean)[0]'''

a,b = train_test_split(t_interp,test_size = 0.90)
dm = np.zeros((len(a),len(a)))

for i in tqdm(range(len(a))):
    
    t0 = np.zeros((len(a[0]),2))
    t0[:,0] = np.arange(len(a[0]))
    t0[:,1] = a[i]
    
    for j in range(len(a)):
        t1 = np.zeros((len(a[0]),2))
        t1[:,0] = np.arange(len(a[0]))
        t1[:,1] = a[j]
        
        #dm[i,j] =similaritymeasures.curve_length_measure(t0,t1)
        dm[i,j] =similaritymeasures.dtw(t0,t1)[0]
        #dm[i,j] =fastdtw(t0,t1,dist = euclidean)[0]

In [ ]:
#np.savetxt(r'C:\Users\kj4755\OneDrive - The Open University\SPIN\Transmission\Clusters\Curve length\New Occultation dm 1.txt',dm)
np.savetxt(r'C:\Users\kj4755\OneDrive - The Open University\SPIN\Transmission\Clusters\DTW\New Occultation 033 dm 1.txt',dm)

In [ ]:
eps = 6
min_samples = 7
clustering = DBSCAN(eps=eps, min_samples=min_samples, metric = 'precomputed').fit(dm)
labels = clustering.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print('Estimated number of clusters: %d' % n_clusters_)

In [ ]:
dm

In [ ]:
labels

In [ ]:
for i in tqdm(set(labels)):
    
    idx = np.array(np.where(labels == i)).flatten()
    #directory = r'C:\Users\kj4755\OneDrive - The Open University\SPIN\Transmission\Clusters\Curve length\eps %s min_samples %s\Cluster %s' %(eps,min_samples,i)
    directory = r'C:\Users\kj4755\OneDrive - The Open University\SPIN\Transmission\Clusters\DTW\eps %s min_samples %s\Cluster %s' %(eps,min_samples,i)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    for _ in idx:
        plt.clf()
        plt.plot(z[_],color = 'k')
        plt.xlabel('Tangent Altitude (km)')
        plt.ylabel('Transmission')
        plt.title('Transmission vs TangAlt at %s nm' %f_wav)
        plt.savefig(directory + '\%s.png' %list_of_files[_])
    
    np.savetxt(directory + '\\Cluster %s files.txt' %i,list_of_files[idx],fmt = '%s')